In [1]:
import pynq
import collections

from pynq import Overlay, UnsupportedConfiguration
from pynq import Bitstream
from pprint import pprint
import numpy as np
from tqdm import tqdm
import time
import re

ol = Overlay('./system.bit')
ol.download()
ol.is_loaded()
#pprint(ol.ip_dict)

True

In [2]:
def pblock_n5():
    
    ####################################### Loda Cardio #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./cardio_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 1831 # X_train.shape[0]
    DIM = 21
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_loda_cardio_c1.bit','s_axi_AXILiteS', 'loda_cardio_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_loda_cardio_c2.bit','s_axi_AXILiteS', 'loda_cardio_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_loda_cardio_c3.bit','s_axi_AXILiteS', 'loda_cardio_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_loda_cardio_c4.bit','s_axi_AXILiteS', 'loda_cardio_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_loda_cardio_c5.bit','s_axi_AXILiteS', 'loda_cardio_c5',None)
    decouper_5.write(0x0,0)
    
    decouper_combo1 = ol.dfx_decoupler_combo1
    decouper_combo1.write(0x0,0)
    decouper_combo1.write(0x0,1)
    ol.pr_download('pr_combo_1','pr_combo_1_combo_score_i3.bit','s_axi_AXILiteS', 'combo_score_i3',None)
    decouper_combo1.write(0x0,0)
    
    decouper_combo2 = ol.dfx_decoupler_combo2
    decouper_combo2.write(0x0,0)
    decouper_combo2.write(0x0,1)
    ol.pr_download('pr_combo_2','pr_combo_2_combo_score_i2.bit','s_axi_AXILiteS', 'combo_score_i2',None)
    decouper_combo2.write(0x0,0)
    
    decouper_combo3 = ol.dfx_decoupler_combo3
    decouper_combo3.write(0x0,0)
    decouper_combo3.write(0x0,1)
    ol.pr_download('pr_combo_3','pr_combo_3_combo_score_i2.bit','s_axi_AXILiteS', 'combo_score_i2',None)
    decouper_combo3.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.loda_cardio_c1
    ad_ip2 = ol.pr2.loda_cardio_c2
    ad_ip3 = ol.pr3.loda_cardio_c3
    ad_ip4 = ol.pr4.loda_cardio_c4
    ad_ip5 = ol.pr5.loda_cardio_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> Loda Cardio 5-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### RS-Hash Cardio #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./cardio_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 1831 # X_train.shape[0]
    DIM = 21
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_rshash_cardio_c1.bit','loda_cardio_c1', 'rshash_cardio_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_rshash_cardio_c2.bit','loda_cardio_c2', 'rshash_cardio_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_rshash_cardio_c3.bit','loda_cardio_c3', 'rshash_cardio_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_rshash_cardio_c4.bit','loda_cardio_c4', 'rshash_cardio_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_rshash_cardio_c5.bit','loda_cardio_c5', 'rshash_cardio_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.rshash_cardio_c1
    ad_ip2 = ol.pr2.rshash_cardio_c2
    ad_ip3 = ol.pr3.rshash_cardio_c3
    ad_ip4 = ol.pr4.rshash_cardio_c4
    ad_ip5 = ol.pr5.rshash_cardio_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> RS-Hash Cardio 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### xStream Cardio #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./cardio_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 1831 # X_train.shape[0]
    DIM = 21
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_xstream_cardio_c1.bit','rshash_cardio_c1', 'xstream_cardio_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_xstream_cardio_c2.bit','rshash_cardio_c2', 'xstream_cardio_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_xstream_cardio_c3.bit','rshash_cardio_c3', 'xstream_cardio_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_xstream_cardio_c4.bit','rshash_cardio_c4', 'xstream_cardio_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_xstream_cardio_c5.bit','rshash_cardio_c5', 'xstream_cardio_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.xstream_cardio_c1
    ad_ip2 = ol.pr2.xstream_cardio_c2
    ad_ip3 = ol.pr3.xstream_cardio_c3
    ad_ip4 = ol.pr4.xstream_cardio_c4
    ad_ip5 = ol.pr5.xstream_cardio_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> xStream Cardio 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### Loda Shuttle #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./shuttle_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 49097 # X_train.shape[0]
    DIM = 9
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    
    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_loda_shuttle_c1.bit','xstream_cardio_c1', 'loda_shuttle_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_loda_shuttle_c2.bit','xstream_cardio_c2', 'loda_shuttle_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_loda_shuttle_c3.bit','xstream_cardio_c3', 'loda_shuttle_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_loda_shuttle_c4.bit','xstream_cardio_c4', 'loda_shuttle_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_loda_shuttle_c5.bit','xstream_cardio_c5', 'loda_shuttle_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.loda_shuttle_c1
    ad_ip2 = ol.pr2.loda_shuttle_c2
    ad_ip3 = ol.pr3.loda_shuttle_c3
    ad_ip4 = ol.pr4.loda_shuttle_c4
    ad_ip5 = ol.pr5.loda_shuttle_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> Loda Shuttle 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### RS-Hash Shuttle #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./shuttle_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 49097 # X_train.shape[0]
    DIM = 9
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_rshash_shuttle_c1.bit','loda_shuttle_c1', 'rshash_shuttle_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_rshash_shuttle_c2.bit','loda_shuttle_c2', 'rshash_shuttle_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_rshash_shuttle_c3.bit','loda_shuttle_c3', 'rshash_shuttle_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_rshash_shuttle_c4.bit','loda_shuttle_c4', 'rshash_shuttle_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_rshash_shuttle_c5.bit','loda_shuttle_c5', 'rshash_shuttle_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.rshash_shuttle_c1
    ad_ip2 = ol.pr2.rshash_shuttle_c2
    ad_ip3 = ol.pr3.rshash_shuttle_c3
    ad_ip4 = ol.pr4.rshash_shuttle_c4
    ad_ip5 = ol.pr5.rshash_shuttle_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> RS-Hash Shuttle 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### xStream Shuttle #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./shuttle_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 49097 # X_train.shape[0]
    DIM = 9
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_xstream_shuttle_c1.bit','rshash_shuttle_c1', 'xstream_shuttle_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_xstream_shuttle_c2.bit','rshash_shuttle_c2', 'xstream_shuttle_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_xstream_shuttle_c3.bit','rshash_shuttle_c3', 'xstream_shuttle_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_xstream_shuttle_c4.bit','rshash_shuttle_c4', 'xstream_shuttle_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_xstream_shuttle_c5.bit','rshash_shuttle_c5', 'xstream_shuttle_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.xstream_shuttle_c1
    ad_ip2 = ol.pr2.xstream_shuttle_c2
    ad_ip3 = ol.pr3.xstream_shuttle_c3
    ad_ip4 = ol.pr4.xstream_shuttle_c4
    ad_ip5 = ol.pr5.xstream_shuttle_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> xStream Shuttle 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### Loda smtp3 #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./smtp1_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 95156 # X_train.shape[0]
    DIM = 3
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_loda_smtp1_c1.bit','xstream_shuttle_c1', 'loda_smtp1_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_loda_smtp1_c2.bit','xstream_shuttle_c2', 'loda_smtp1_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_loda_smtp1_c3.bit','xstream_shuttle_c3', 'loda_smtp1_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_loda_smtp1_c4.bit','xstream_shuttle_c4', 'loda_smtp1_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_loda_smtp1_c5.bit','xstream_shuttle_c5', 'loda_smtp1_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.loda_smtp1_c1
    ad_ip2 = ol.pr2.loda_smtp1_c2
    ad_ip3 = ol.pr3.loda_smtp1_c3
    ad_ip4 = ol.pr4.loda_smtp1_c4
    ad_ip5 = ol.pr5.loda_smtp1_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> Loda smtp3 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### RS-Hash smtp3 #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./smtp1_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 95156 # X_train.shape[0]
    DIM = 3
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_rshash_smtp1_c1.bit','loda_smtp1_c1', 'rshash_smtp1_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_rshash_smtp1_c2.bit','loda_smtp1_c2', 'rshash_smtp1_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_rshash_smtp1_c3.bit','loda_smtp1_c3', 'rshash_smtp1_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_rshash_smtp1_c4.bit','loda_smtp1_c4', 'rshash_smtp1_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_rshash_smtp1_c5.bit','loda_smtp1_c5', 'rshash_smtp1_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.rshash_smtp1_c1
    ad_ip2 = ol.pr2.rshash_smtp1_c2
    ad_ip3 = ol.pr3.rshash_smtp1_c3
    ad_ip4 = ol.pr4.rshash_smtp1_c4
    ad_ip5 = ol.pr5.rshash_smtp1_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> Loda smtp3 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### xStream smtp3 #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./smtp1_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 95156 # X_train.shape[0]
    DIM = 3
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_xstream_smtp1_c1.bit','rshash_smtp1_c1', 'xstream_smtp1_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_xstream_smtp1_c2.bit','rshash_smtp1_c2', 'xstream_smtp1_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_xstream_smtp1_c3.bit','rshash_smtp1_c3', 'xstream_smtp1_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_xstream_smtp1_c4.bit','rshash_smtp1_c4', 'xstream_smtp1_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_xstream_smtp1_c5.bit','rshash_smtp1_c5', 'xstream_smtp1_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.xstream_smtp1_c1
    ad_ip2 = ol.pr2.xstream_smtp1_c2
    ad_ip3 = ol.pr3.xstream_smtp1_c3
    ad_ip4 = ol.pr4.xstream_smtp1_c4
    ad_ip5 = ol.pr5.xstream_smtp1_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> xStream smtp3 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### Loda http3 #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./http1_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 567498 # X_train.shape[0]
    DIM = 3
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_loda_http1_c1.bit','xstream_smtp1_c1', 'loda_http1_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_loda_http1_c2.bit','xstream_smtp1_c2', 'loda_http1_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_loda_http1_c3.bit','xstream_smtp1_c3', 'loda_http1_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_loda_http1_c4.bit','xstream_smtp1_c4', 'loda_http1_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_loda_http1_c5.bit','xstream_smtp1_c5', 'loda_http1_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.loda_http1_c1
    ad_ip2 = ol.pr2.loda_http1_c2
    ad_ip3 = ol.pr3.loda_http1_c3
    ad_ip4 = ol.pr4.loda_http1_c4
    ad_ip5 = ol.pr5.loda_http1_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> Loda http3 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### RS-Hash http3 #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./http1_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 567498 # X_train.shape[0]
    DIM = 3
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]
    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_rshash_http1_c1.bit','loda_http1_c1', 'rshash_http1_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_rshash_http1_c2.bit','loda_http1_c2', 'rshash_http1_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_rshash_http1_c3.bit','loda_http1_c3', 'rshash_http1_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_rshash_http1_c4.bit','loda_http1_c4', 'rshash_http1_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_rshash_http1_c5.bit','loda_http1_c5', 'rshash_http1_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.rshash_http1_c1
    ad_ip2 = ol.pr2.rshash_http1_c2
    ad_ip3 = ol.pr3.rshash_http1_c3
    ad_ip4 = ol.pr4.rshash_http1_c4
    ad_ip5 = ol.pr5.rshash_http1_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> RS-Hash http3 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    ####################################### xStream http3 #######################################
    CONTROL_REGISTER = 0x0
    def _load_via_txt(path):
        X = np.loadtxt(path, delimiter=",")
        return X

    def get_data(data_file):
        #data_path = os.path.join(data_base_path, data_file)
        data_path = data_file
        if ".mat" in data_file:
            from scipy.io import loadmat
            f = loadmat(data_path)
            X = f['X']
        else:
            X = _load_via_txt(data_path)
        return X

    X_train = get_data("./http1_X")  # Load Aryhytmia data.
    print("X_train.shape = ",X_train.shape)

    N = 567498 # X_train.shape[0]
    DIM = 3
    buff_x0    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x1    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x2    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x3    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_x4    = pynq.allocate(shape=(N,DIM), dtype=np.float32)
    buff_y0    = pynq.allocate(shape=(N,),  dtype=np.float32)

    buff_x0[:] = X_train[0:N][:]
    buff_x1[:] = X_train[0:N][:]
    buff_x2[:] = X_train[0:N][:]
    buff_x3[:] = X_train[0:N][:]
    buff_x4[:] = X_train[0:N][:]

    decouper_1 = ol.dfx_decoupler_1
    decouper_1.write(0x0,0)
    decouper_1.write(0x0,1)
    ol.pr_download('pr1','pr1_xstream_http1_c1.bit','rshash_http1_c1', 'xstream_http1_c1',None)
    decouper_1.write(0x0,0)
    
    decouper_2 = ol.dfx_decoupler_2
    decouper_2.write(0x0,0)
    decouper_2.write(0x0,1)
    ol.pr_download('pr2','pr2_xstream_http1_c2.bit','rshash_http1_c2', 'xstream_http1_c2',None)
    decouper_2.write(0x0,0)
    
    decouper_3 = ol.dfx_decoupler_3
    decouper_3.write(0x0,0)
    decouper_3.write(0x0,1)
    ol.pr_download('pr3','pr3_xstream_http1_c3.bit','rshash_http1_c3', 'xstream_http1_c3',None)
    decouper_3.write(0x0,0)
    
    decouper_4 = ol.dfx_decoupler_4
    decouper_4.write(0x0,0)
    decouper_4.write(0x0,1)
    ol.pr_download('pr4','pr4_xstream_http1_c4.bit','rshash_http1_c4', 'xstream_http1_c4',None)
    decouper_4.write(0x0,0)
    
    decouper_5 = ol.dfx_decoupler_5
    decouper_5.write(0x0,0)
    decouper_5.write(0x0,1)
    ol.pr_download('pr5','pr5_xstream_http1_c5.bit','rshash_http1_c5', 'xstream_http1_c5',None)
    decouper_5.write(0x0,0)
    
    combo_ip1 = ol.pr_combo_1.combo_score_i3
    combo_ip2 = ol.pr_combo_2.combo_score_i2
    combo_ip3 = ol.pr_combo_3.combo_score_i2
    ad_ip1 = ol.pr1.xstream_http1_c1
    ad_ip2 = ol.pr2.xstream_http1_c2
    ad_ip3 = ol.pr3.xstream_http1_c3
    ad_ip4 = ol.pr4.xstream_http1_c4
    ad_ip5 = ol.pr5.xstream_http1_c5
    
    axiswitch_ip1 = ol.axis_switch
    axiswitch_ip2 = ol.axis_switch1

    dma_x0 = ol.axi_dma_in0.sendchannel
    dma_x1 = ol.axi_dma_in1.sendchannel
    dma_x2 = ol.axi_dma_in2.sendchannel
    dma_x3 = ol.axi_dma_in3.sendchannel
    dma_x4 = ol.axi_dma_in4.sendchannel
    dma_y0 = ol.axi_dma_in0.recvchannel


    axiswitch_ip1.write(0x40, 0x00) # m0
    axiswitch_ip1.write(0x44, 0x01) # m1
    axiswitch_ip1.write(0x48, 0x02) # m2
    axiswitch_ip1.write(0x4c, 0x03) # m3
    axiswitch_ip1.write(0x50, 0x04) # m4
    axiswitch_ip1.write(0x5c, 0x07) # m7
    
    axiswitch_ip2.write(0x40, 0x00) # m0
    axiswitch_ip2.write(0x44, 0x01) # m1
    axiswitch_ip2.write(0x48, 0x02) # m2
    
    axiswitch_ip2.write(0x50, 0x03) # m4
    axiswitch_ip2.write(0x54, 0x04) # m5
    
    axiswitch_ip2.write(0x60, 0x07) # m8
    axiswitch_ip2.write(0x64, 0x08) # m9
    
    axiswitch_ip2.write(0x70, 0x09) # m12

    axiswitch_ip1.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1
    axiswitch_ip2.write(CONTROL_REGISTER, 0x02) # 0x02 will set bit 1

    ad_ip1.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip2.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip3.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip4.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    ad_ip5.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
    combo_ip1.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip2.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    combo_ip3.write(CONTROL_REGISTER, 0x81) # 0x02 will set bit 1
    
    start_ad = time.time()
    dma_x0.transfer(buff_x0)
    dma_x1.transfer(buff_x1)
    dma_x2.transfer(buff_x2)
    dma_x3.transfer(buff_x3)
    dma_x4.transfer(buff_x4)
    dma_y0.transfer(buff_y0)
    dma_x0.wait()
    dma_x1.wait()
    dma_x2.wait()
    dma_x3.wait()
    dma_x4.wait()
    dma_y0.wait()

    end_ad = time.time()
    ad_ip1.write(CONTROL_REGISTER,0x00) # stop
    ad_ip2.write(CONTROL_REGISTER,0x00) # stop
    ad_ip3.write(CONTROL_REGISTER,0x00) # stop
    ad_ip4.write(CONTROL_REGISTER,0x00) # stop
    ad_ip5.write(CONTROL_REGISTER,0x00) # stop
    combo_ip1.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip2.write(CONTROL_REGISTER, 0x00) # stop
    combo_ip3.write(CONTROL_REGISTER, 0x00) # stop
    time_cost= end_ad-start_ad
    print('=> xStream http3 4-pblock')
    print('timecost(ms) = ',time_cost*1000)
    print('')
    del buff_x0, buff_x1, buff_x2, buff_x3, buff_x4, buff_y0
    
    print('Successful!')

In [3]:
pblock_n5()

X_train.shape =  (1831, 21)
=> Loda Cardio 5-pblock
timecost(ms) =  4.282951354980469

X_train.shape =  (1831, 21)
=> RS-Hash Cardio 4-pblock
timecost(ms) =  4.458427429199219

X_train.shape =  (1831, 21)
=> xStream Cardio 4-pblock
timecost(ms) =  4.5146942138671875

X_train.shape =  (49097, 9)
=> Loda Shuttle 4-pblock
timecost(ms) =  33.94818305969238

X_train.shape =  (49097, 9)
=> RS-Hash Shuttle 4-pblock
timecost(ms) =  35.53509712219238

X_train.shape =  (49097, 9)
=> xStream Shuttle 4-pblock
timecost(ms) =  40.17472267150879

X_train.shape =  (95156, 3)
=> Loda smtp3 4-pblock
timecost(ms) =  39.00289535522461

X_train.shape =  (95156, 3)
=> Loda smtp3 4-pblock
timecost(ms) =  39.00718688964844

X_train.shape =  (95156, 3)
=> xStream smtp3 4-pblock
timecost(ms) =  50.67181587219238

X_train.shape =  (567498, 3)
=> Loda http3 4-pblock
timecost(ms) =  227.95677185058594

X_train.shape =  (567498, 3)
=> RS-Hash http3 4-pblock
timecost(ms) =  227.9338836669922

X_train.shape =  (56749